In [1]:
input_dir = '../input/'
working_dir = '../working/'

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
train = pd.read_csv(os.path.join(input_dir, 'train.csv'))
test = pd.read_csv(os.path.join(input_dir, 'test.csv'))

# Set index
train.index = train['Id'].values
test.index = test['Id'].values

print(train.shape)
print(test.shape)

(9557, 143)
(23856, 142)


# **EDA**

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9557 entries, ID_279628684 to ID_a38c64491
Columns: 143 entries, Id to Target
dtypes: float64(8), int64(130), object(5)
memory usage: 10.5+ MB


In [5]:
train.head(5)

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,r4h3,r4m1,r4m2,r4m3,r4t1,r4t2,r4t3,tamhog,tamviv,escolari,rez_esc,hhsize,paredblolad,paredzocalo,paredpreb,pareddes,paredmad,paredzinc,paredfibras,paredother,pisomoscer,pisocemento,pisoother,pisonatur,pisonotiene,pisomadera,techozinc,techoentrepiso,techocane,...,meaneduc,instlevel1,instlevel2,instlevel3,instlevel4,instlevel5,instlevel6,instlevel7,instlevel8,instlevel9,bedrooms,overcrowding,tipovivi1,tipovivi2,tipovivi3,tipovivi4,tipovivi5,computer,television,mobilephone,qmobilephone,lugar1,lugar2,lugar3,lugar4,lugar5,lugar6,area1,area2,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
ID_279628684,ID_279628684,190000.0,0,3,0,1,1,0,NaN,0,1,1,0,0,0,0,1,1,1,1,10,NaN,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,...,10.0,0,0,0,1,0,0,0,0,0,1,1.000000,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,1,0,43,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
ID_f29eb3ddd,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,1,1,0,0,0,0,1,1,1,1,12,NaN,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,...,12.0,0,0,0,0,0,0,0,1,0,1,1.000000,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,1,0,67,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
ID_68de51c94,ID_68de51c94,NaN,0,8,0,1,1,0,NaN,0,0,0,0,1,1,0,1,1,1,1,11,NaN,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,...,11.0,0,0,0,0,1,0,0,0,0,2,0.500000,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,92,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
ID_d671db89c,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,2,2,1,1,2,1,3,4,4,4,9,1.0,4,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,11.0,0,0,0,1,0,0,0,0,0,3,1.333333,0,0,1,0,0,0,0,1,3,1,0,0,0,0,0,1,0,17,81,289,16,121,4,1.777778,1.0,121.0,289,4
ID_d56d6f5f5,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,2,2,1,1,2,1,3,4,4,4,11,NaN,4,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,11.0,0,0,0,0,1,0,0,0,0,3,1.333333,0,0,1,0,0,0,0,1,3,1,0,0,0,0,0,1,0,37,121,1369,16,121,4,1.777778,1.0,121.0,1369,4


The data have 9557 entries, each entry has 143 columns.

Most of the data are floats and integers, a few objects. Let's take a look at the objects.

# **Object value**

In [6]:
train.columns[train.dtypes==object]

Index(['Id', 'idhogar', 'dependency', 'edjefe', 'edjefa'], dtype='object')

Id, idhogar - no problem, they are just identifications


dependency - dependency rate


edjefe, edjefa - years of education of head of household



### Clean Data


1. dependency 'no' -> 0
2. edjefa, edjefe 'no' -> 0,  'yes' -> 1
3. meaneduc NaN -> mean escolari of household
4. v2a1 NaN -> 0
5. v18q1 NaN -> 0
6. rez_esc NaN -> 0

1. dependency

dependency 'no' -> 0

we can just derive the dependency from the SQBdependency.

* So the "square" of no is 0.

* So the "square" of yes is 1.

In [7]:
train['dependency'].unique()

array(['no', '8', 'yes', '3', '.5', '.25', '2', '.66666669', '.33333334',
       '1.5', '.40000001', '.75', '1.25', '.2', '2.5', '1.2', '4',
       '1.3333334', '2.25', '.22222222', '5', '.83333331', '.80000001',
       '6', '3.5', '1.6666666', '.2857143', '1.75', '.71428573',
       '.16666667', '.60000002'], dtype=object)

In [8]:
train['SQBdependency'].unique()

array([0.0000000e+00, 6.4000000e+01, 1.0000000e+00, 9.0000000e+00,
       2.5000000e-01, 6.2500000e-02, 4.0000000e+00, 4.4444448e-01,
       1.1111112e-01, 2.2500000e+00, 1.6000001e-01, 5.6250000e-01,
       1.5625000e+00, 4.0000003e-02, 6.2500000e+00, 1.4400001e+00,
       1.6000000e+01, 1.7777779e+00, 5.0625000e+00, 4.9382716e-02,
       2.5000000e+01, 6.9444442e-01, 6.4000005e-01, 3.6000000e+01,
       1.2250000e+01, 2.7777777e+00, 8.1632659e-02, 3.0625000e+00,
       5.1020408e-01, 2.7777780e-02, 3.6000001e-01])

In [9]:
train['SQBdependency']

ID_279628684     0.000000
ID_f29eb3ddd    64.000000
ID_68de51c94    64.000000
ID_d671db89c     1.000000
ID_d56d6f5f5     1.000000
ID_ec05b1a7b     1.000000
ID_e9e0c1100     1.000000
ID_3e04e571e     1.000000
ID_1284f8aad     1.000000
ID_51f52fdd2     1.000000
ID_db44f5c59     1.000000
ID_de822510c     1.000000
ID_d94071d7c     1.000000
ID_064b57869     1.000000
ID_5c837d8a4     1.000000
ID_0a39e419e     9.000000
ID_4ff51f90c     9.000000
ID_336c51386     9.000000
ID_c51938edf     9.000000
ID_35b66f7c6     0.000000
ID_74a2bc523     0.000000
ID_15d393ce7    64.000000
ID_a0bff0ba7    64.000000
ID_848b0fdf1     0.000000
ID_298d857e5     0.000000
ID_1d5bf8ddf     0.000000
ID_e5cdba865     0.000000
ID_e18530139     0.250000
ID_cba035c44     0.250000
ID_b6c1c7e65     0.250000
                  ...    
ID_92b30d94b     1.000000
ID_6309f1595     0.000000
ID_f8b8c7703     0.000000
ID_7226bda20     0.000000
ID_10da2c3cb     0.250000
ID_503cfc817     0.250000
ID_5384717d3     0.250000
ID_8d3963fff

In [10]:
train[(train['dependency']=='no') & (train['SQBdependency']!=0)]

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,r4h3,r4m1,r4m2,r4m3,r4t1,r4t2,r4t3,tamhog,tamviv,escolari,rez_esc,hhsize,paredblolad,paredzocalo,paredpreb,pareddes,paredmad,paredzinc,paredfibras,paredother,pisomoscer,pisocemento,pisoother,pisonatur,pisonotiene,pisomadera,techozinc,techoentrepiso,techocane,...,meaneduc,instlevel1,instlevel2,instlevel3,instlevel4,instlevel5,instlevel6,instlevel7,instlevel8,instlevel9,bedrooms,overcrowding,tipovivi1,tipovivi2,tipovivi3,tipovivi4,tipovivi5,computer,television,mobilephone,qmobilephone,lugar1,lugar2,lugar3,lugar4,lugar5,lugar6,area1,area2,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target


In [11]:
train[(train['dependency']=='yes') & (train['SQBdependency']!=1)]

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,r4h3,r4m1,r4m2,r4m3,r4t1,r4t2,r4t3,tamhog,tamviv,escolari,rez_esc,hhsize,paredblolad,paredzocalo,paredpreb,pareddes,paredmad,paredzinc,paredfibras,paredother,pisomoscer,pisocemento,pisoother,pisonatur,pisonotiene,pisomadera,techozinc,techoentrepiso,techocane,...,meaneduc,instlevel1,instlevel2,instlevel3,instlevel4,instlevel5,instlevel6,instlevel7,instlevel8,instlevel9,bedrooms,overcrowding,tipovivi1,tipovivi2,tipovivi3,tipovivi4,tipovivi5,computer,television,mobilephone,qmobilephone,lugar1,lugar2,lugar3,lugar4,lugar5,lugar6,area1,area2,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target


In [12]:
train[(train['dependency']=='no') & (train['SQBdependency']!=1)]

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,r4h3,r4m1,r4m2,r4m3,r4t1,r4t2,r4t3,tamhog,tamviv,escolari,rez_esc,hhsize,paredblolad,paredzocalo,paredpreb,pareddes,paredmad,paredzinc,paredfibras,paredother,pisomoscer,pisocemento,pisoother,pisonatur,pisonotiene,pisomadera,techozinc,techoentrepiso,techocane,...,meaneduc,instlevel1,instlevel2,instlevel3,instlevel4,instlevel5,instlevel6,instlevel7,instlevel8,instlevel9,bedrooms,overcrowding,tipovivi1,tipovivi2,tipovivi3,tipovivi4,tipovivi5,computer,television,mobilephone,qmobilephone,lugar1,lugar2,lugar3,lugar4,lugar5,lugar6,area1,area2,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
ID_279628684,ID_279628684,190000.0,0,3,0,1,1,0,NaN,0,1,1,0,0,0,0,1,1,1,1,10,NaN,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,...,10.000000,0,0,0,1,0,0,0,0,0,1,1.000000,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,1,0,43,100,1849,1,100,0,1.000000,0.0,100.000000,1849,4
ID_35b66f7c6,ID_35b66f7c6,215000.0,0,4,0,1,1,0,NaN,0,1,1,0,1,1,0,2,2,2,2,12,NaN,2,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,13.500000,0,0,0,0,0,0,1,0,0,2,1.000000,0,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,22,144,484,4,225,0,1.000000,0.0,182.250000,484,4
ID_74a2bc523,ID_74a2bc523,215000.0,0,4,0,1,1,0,NaN,0,1,1,0,1,1,0,2,2,2,2,15,NaN,2,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,13.500000,0,0,0,0,0,0,0,1,0,2,1.000000,0,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,26,225,676,4,225,0,1.000000,0.0,182.250000,676,4
ID_848b0fdf1,ID_848b0fdf1,100000.0,1,1,1,0,1,0,NaN,0,2,2,0,1,1,0,3,3,3,3,6,NaN,3,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,10.333333,0,0,1,0,0,0,0,0,0,1,3.000000,0,0,1,0,0,0,0,1,3,1,0,0,0,0,0,1,0,41,36,1681,9,36,0,9.000000,0.0,106.777770,1681,4
ID_298d857e5,ID_298d857e5,100000.0,1,1,1,0,1,0,NaN,0,2,2,0,1,1,0,3,3,3,3,13,NaN,3,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,10.333333,0,0,0,0,0,0,0,1,0,1,3.000000,0,0,1,0,0,0,0,1,3,1,0,0,0,0,0,1,0,20,169,400,9,36,0,9.000000,0.0,106.777770,400,4
ID_1d5bf8ddf,ID_1d5bf8ddf,100000.0,1,1,1,0,1,0,NaN,0,2,2,0,1,1,0,3,3,3,3,12,NaN,3,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,10.333333,0,0,0,0,0,0,1,0,0,1,3.000000,0,0,1,0,0,0,0,1,3,1,0,0,0,0,0,1,0,40,144,1600,9,36,0,9.000000,0.0,106.777770,1600,4
ID_e5cdba865,ID_e5cdba865,NaN,0,5,0,1,1,0,NaN,0,1,1,0,0,0,0,1,1,1,1,15,NaN,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,...,15.000000,0,0,0,0,0,0,0,1,0,2,0.500000,1,0,0,0,0,0,0,1,2,1,0,0,0,0,0,1,0,44,225,1936,1,225,0,0.250000,0.0,225.000000,1936,4
ID_96b1fbc81,ID_96b1fbc81,100000.0,0,5,0,1,1,0,NaN,0,1,1,0,1,1,0,2,2,2,2,8,NaN,2,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,9.500000,0,0,0,1,0,0,0,0,0,2,1.000000,0,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,35,64,1225,4,64,0,1.000000,0.0,90.250000,1225,4
ID_422226730,ID_422226730,100000.0,0,5,0,1,1,0,NaN,0,1,1,0,1,1,0,2,2,2,2,11,NaN,2,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,9.500000,0,0,0,0,1,0,0,0,0,2,1.000000,0,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,28,121,784,4,64,0,1.000000,0.0,90.250000,784,4
ID_488b8ac27,ID_488b8ac27,90000.0,0,5,0,1,1,1,1.0,0,1,1,0,1,1,0,2,2,2,2,9,NaN,2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,...,11.000000,0,0,0,1,0,0,0,0,0,2,1.000000,0,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,44,81,1936,4,0,0,1.000000,0.0,121.000000,1936,4


In [13]:
train['dependency']=np.sqrt(train['SQBdependency'])

2. edjefa, edjefe 'no' -> 0, 'yes' -> 1

Basically:

* 'edjefe' and 'edjefa' are both 'no' when the head of the household had 0 years of school
* there's 'edjefe'= 'yes' and 'edjefa'='no' in some cases, all these cases the head of the household had 1 year of school
* there's 'edjefe'= 'no' and 'edjefa'='yes' in some cases, all these cases the head of the household had 1 year of school
* most of the time either 'edjefe' or 'edjefa' is a number while the other is a 'no'
* Let's merge the jefe and jefa education into one, undependent of gender

In [14]:
train['edjefa'].unique()

array(['no', '11', '4', '10', '9', '15', '7', '14', '13', '8', '17', '6',
       '5', '3', '16', '19', 'yes', '21', '12', '2', '20', '18'],
      dtype=object)

In [15]:
train['edjefa'].unique()

array(['no', '11', '4', '10', '9', '15', '7', '14', '13', '8', '17', '6',
       '5', '3', '16', '19', 'yes', '21', '12', '2', '20', '18'],
      dtype=object)

In [16]:
train['SQBedjefe'].unique()

array([100, 144,   0, 121,  81, 225,  16,  36,  64, 289,  49, 256, 196,
        25, 441,   4, 361,   1,   9, 324, 169, 400])

In [17]:
train[['edjefe', 'edjefa', 'SQBedjefe']][:20]

,edjefe,edjefa,SQBedjefe
ID_279628684,10,no,100
ID_f29eb3ddd,12,no,144
ID_68de51c94,no,11,0
ID_d671db89c,11,no,121
ID_d56d6f5f5,11,no,121
ID_ec05b1a7b,11,no,121
ID_e9e0c1100,11,no,121
ID_3e04e571e,9,no,81
ID_1284f8aad,9,no,81
ID_51f52fdd2,9,no,81


'SQBedjefe is just the square of 'edjefe', it's 0 if the head of the household is a woman.

In [18]:
train[['edjefe', 'edjefa', 'SQBedjefe']][train['edjefe']=='yes']

,edjefe,edjefa,SQBedjefe
ID_3ffc11a03,yes,no,1
ID_76cbd395d,yes,no,1
ID_cb914fc3b,yes,no,1
ID_48b1e8fb0,yes,no,1
ID_bdf3d8862,yes,no,1
ID_6ec7ee687,yes,no,1
ID_a1a5f5e09,yes,no,1
ID_21ff47f96,yes,no,1
ID_e5ee8c82f,yes,no,1
ID_9ac731b14,yes,no,1


In [19]:
train[(train['edjefe']=='yes') & (train['edjefa']!='no')]

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,r4h3,r4m1,r4m2,r4m3,r4t1,r4t2,r4t3,tamhog,tamviv,escolari,rez_esc,hhsize,paredblolad,paredzocalo,paredpreb,pareddes,paredmad,paredzinc,paredfibras,paredother,pisomoscer,pisocemento,pisoother,pisonatur,pisonotiene,pisomadera,techozinc,techoentrepiso,techocane,...,meaneduc,instlevel1,instlevel2,instlevel3,instlevel4,instlevel5,instlevel6,instlevel7,instlevel8,instlevel9,bedrooms,overcrowding,tipovivi1,tipovivi2,tipovivi3,tipovivi4,tipovivi5,computer,television,mobilephone,qmobilephone,lugar1,lugar2,lugar3,lugar4,lugar5,lugar6,area1,area2,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target


escolari = years of schooling

parentesco1 =1 if household head

In [20]:
train[(train['edjefa']=='yes') & (train['parentesco1']==1)][['edjefe', 'edjefa', 'parentesco1', 'escolari']]

,edjefe,edjefa,parentesco1,escolari
ID_1b32caf34,no,yes,1,1
ID_f23811c6b,no,yes,1,1
ID_68ec000c5,no,yes,1,1
ID_2d7e7d8d1,no,yes,1,1
ID_e7cd87c9a,no,yes,1,1
ID_5e05f1f28,no,yes,1,1
ID_7cdd89568,no,yes,1,1
ID_83a8404fa,no,yes,1,1
ID_d3b16fb72,no,yes,1,1
ID_bce554d9d,no,yes,1,1


In [21]:
train[train['edjefe']=='yes'][['edjefe', 'edjefa','age', 'escolari', 'parentesco1','male', 'female', 'idhogar']]

,edjefe,edjefa,age,escolari,parentesco1,male,female,idhogar
ID_3ffc11a03,yes,no,81,1,1,1,0,3641ce2d1
ID_76cbd395d,yes,no,73,3,0,0,1,3641ce2d1
ID_cb914fc3b,yes,no,42,6,0,0,1,a7abf59cc
ID_48b1e8fb0,yes,no,12,5,0,0,1,a7abf59cc
ID_bdf3d8862,yes,no,46,1,1,1,0,a7abf59cc
ID_6ec7ee687,yes,no,41,6,0,1,0,698318dd7
ID_a1a5f5e09,yes,no,74,1,0,0,1,698318dd7
ID_21ff47f96,yes,no,78,1,1,1,0,698318dd7
ID_e5ee8c82f,yes,no,72,4,0,0,1,4559a5af0
ID_9ac731b14,yes,no,76,1,1,1,0,4559a5af0


In [22]:
train[(train['edjefe']=='no') & (train['edjefa']=='no')][['edjefe', 'edjefa', 'age', 'escolari', 'female', 'male', 'Id', 'parentesco1', 'idhogar']]

,edjefe,edjefa,age,escolari,female,male,Id,parentesco1,idhogar
ID_48c10ba80,no,no,89,0,1,0,ID_48c10ba80,1,2b1a06ddc
ID_1a283d51b,no,no,55,13,0,1,ID_1a283d51b,0,2b1a06ddc
ID_fdf18bbee,no,no,21,14,1,0,ID_fdf18bbee,0,d9e9b591a
ID_cba836ed8,no,no,47,8,0,1,ID_cba836ed8,0,d9e9b591a
ID_6031cb88d,no,no,49,9,1,0,ID_6031cb88d,0,d9e9b591a
ID_0bc817cb2,no,no,68,0,1,0,ID_0bc817cb2,1,d9e9b591a
ID_1cfb76926,no,no,21,14,1,0,ID_1cfb76926,0,d9e9b591a
ID_f435c5e46,no,no,6,0,1,0,ID_f435c5e46,0,71bec40bc
ID_a51262dae,no,no,9,1,0,1,ID_a51262dae,0,71bec40bc
ID_7e6d23b28,no,no,26,5,1,0,ID_7e6d23b28,0,71bec40bc


In [23]:
conditions = [
    (train['edjefe']=='no') & (train['edjefa']=='no'), #both no
    (train['edjefe']=='yes') & (train['edjefa']=='no'), # yes and no
    (train['edjefe']=='no') & (train['edjefa']=='yes'), #no and yes 
    (train['edjefe']!='no') & (train['edjefe']!='yes') & (train['edjefa']=='no'), # number and no
    (train['edjefe']=='no') & (train['edjefa']!='no') # no and number
]
choices = [0, 1, 1, train['edjefe'], train['edjefa']]
train['edjefx']=np.select(conditions, choices)
train['edjefx']=train['edjefx'].astype(int)
train[['edjefe', 'edjefa', 'edjefx']][:15]

,edjefe,edjefa,edjefx
ID_279628684,10,no,10
ID_f29eb3ddd,12,no,12
ID_68de51c94,no,11,11
ID_d671db89c,11,no,11
ID_d56d6f5f5,11,no,11
ID_ec05b1a7b,11,no,11
ID_e9e0c1100,11,no,11
ID_3e04e571e,9,no,9
ID_1284f8aad,9,no,9
ID_51f52fdd2,9,no,9


# **missing values**

In [24]:
train.columns[train.isna().sum()!=0]

Index(['v2a1', 'v18q1', 'rez_esc', 'meaneduc', 'SQBmeaned'], dtype='object')

Columns with nans:

* v2a1 - monthly rent
* v18q1 - number of tablets
* rez_esc - years behind school
* meaneduc - mean education for adults
* SQBmeaned - square of meaned

3. meaneduc NaN -> mean escolari of household

'meaneduc' and 'SQBmeaned' are related

In [25]:
train[train['meaneduc'].isnull()]

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,r4h3,r4m1,r4m2,r4m3,r4t1,r4t2,r4t3,tamhog,tamviv,escolari,rez_esc,hhsize,paredblolad,paredzocalo,paredpreb,pareddes,paredmad,paredzinc,paredfibras,paredother,pisomoscer,pisocemento,pisoother,pisonatur,pisonotiene,pisomadera,techozinc,techoentrepiso,techocane,...,instlevel1,instlevel2,instlevel3,instlevel4,instlevel5,instlevel6,instlevel7,instlevel8,instlevel9,bedrooms,overcrowding,tipovivi1,tipovivi2,tipovivi3,tipovivi4,tipovivi5,computer,television,mobilephone,qmobilephone,lugar1,lugar2,lugar3,lugar4,lugar5,lugar6,area1,area2,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target,edjefx
ID_bd8e11b0f,ID_bd8e11b0f,NaN,0,7,0,1,1,0,NaN,0,0,0,0,1,1,0,1,1,1,1,10,NaN,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,5,0.2,1,0,0,0,0,1,1,1,1,1,0,0,0,0,0,1,0,18,100,324,1,0,1,0.04,64.0,NaN,324,4,10
ID_46ff87316,ID_46ff87316,110000.0,0,1,0,1,1,0,NaN,0,1,1,0,1,1,0,2,2,2,2,6,NaN,2,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1,2.0,0,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,18,36,324,4,16,4,4.00,64.0,NaN,324,4,4
ID_69f50bf3e,ID_69f50bf3e,110000.0,0,1,0,1,1,0,NaN,0,1,1,0,1,1,0,2,2,2,2,4,NaN,2,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1,2.0,0,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,18,16,324,4,16,4,4.00,64.0,NaN,324,4,4
ID_db3168f9f,ID_db3168f9f,180000.0,0,3,0,1,1,0,NaN,0,2,2,0,0,0,0,2,2,2,2,12,NaN,2,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,2,1.0,0,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,19,144,361,4,144,4,1.00,64.0,NaN,361,4,12
ID_2a7615902,ID_2a7615902,180000.0,0,3,0,1,1,0,NaN,0,2,2,0,0,0,0,2,2,2,2,12,NaN,2,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,2,1.0,0,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,19,144,361,4,144,4,1.00,64.0,NaN,361,4,12


In [26]:
train[train['meaneduc'].isnull()][['Id','idhogar','edjefe','edjefa', 'hogar_adul', 'hogar_mayor', 'hogar_nin', 'age', 'escolari']]

,Id,idhogar,edjefe,edjefa,hogar_adul,hogar_mayor,hogar_nin,age,escolari
ID_bd8e11b0f,ID_bd8e11b0f,1b31fd159,no,10,0,0,1,18,10
ID_46ff87316,ID_46ff87316,a874b7ce7,4,no,0,0,2,18,6
ID_69f50bf3e,ID_69f50bf3e,a874b7ce7,4,no,0,0,2,18,4
ID_db3168f9f,ID_db3168f9f,faaebf71a,12,no,0,0,2,19,12
ID_2a7615902,ID_2a7615902,faaebf71a,12,no,0,0,2,19,12


So, the 5 rows with Nan for 'meaneduc' is just 3 households, where 18-19 year-olds live. No other people live in these households. Then we can just take the education levels of these kids ('escolari') and put them into 'meaneduc' and 'SQBmeaned'.

4. v2a1 NaN -> 0

Next, let's look at 'v2a1', the monthly rent payment, that also has missing values.

In [27]:
norent=train[train['v2a1'].isnull()]
print("Owns his house:", norent[norent['tipovivi1']==1]['Id'].count())
print("Owns his house paying installments", norent[norent['tipovivi2']==1]['Id'].count())
print("Rented ", norent[norent['tipovivi3']==1]['Id'].count())
print("Precarious ", norent[norent['tipovivi4']==1]['Id'].count())
print("Other ", norent[norent['tipovivi5']==1]['Id'].count())
print("Total ", 6860)

Owns his house: 5911
Owns his house paying installments 0
Rented  0
Precarious  163
Other  786
Total  6860


The majority in fact owns their houses, only a few have odd situations. We can probably just assume they don't pay rent, and put 0 in these cases.

5. v18q1 NaN -> 0

let's look at 'v18q1', which indicates how many tablets the household owns.

In [28]:
train['v18q1'].unique()

array([nan,  1.,  2.,  3.,  4.,  5.,  6.])

6. rez_esc NaN -> 0

rez_esc
 : Years behind in school

In [29]:
rez_esc_nan=train[train['rez_esc'].isnull()]
rez_esc_nan[(rez_esc_nan['age']<18) & rez_esc_nan['escolari']>0][['age', 'escolari']]

,age,escolari


So all the nans here are either adults or children before school age. We can input 0 again

In [30]:
def data_cleaning(data):
    data['dependency']=np.sqrt(data['SQBdependency'])
    data['rez_esc']=data['rez_esc'].fillna(0)
    data['v18q1']=data['v18q1'].fillna(0)
    data['v2a1']=data['v2a1'].fillna(0)
    
    conditions = [
    (data['edjefe']=='no') & (data['edjefa']=='no'), #both no
    (data['edjefe']=='yes') & (data['edjefa']=='no'), # yes and no
    (data['edjefe']=='no') & (data['edjefa']=='yes'), #no and yes 
    (data['edjefe']!='no') & (data['edjefe']!='yes') & (data['edjefa']=='no'), # number and no
    (data['edjefe']=='no') & (data['edjefa']!='no') # no and number
    ]
    choices = [0, 1, 1, data['edjefe'], data['edjefa']]
    data['edjefx']=np.select(conditions, choices)
    data['edjefx']=data['edjefx'].astype(int)
    data.drop(['edjefe', 'edjefa'], axis=1, inplace=True)
    
    meaneduc_nan=data[data['meaneduc'].isnull()][['Id','idhogar','escolari']]
    me=meaneduc_nan.groupby('idhogar')['escolari'].mean().reset_index()
    for row in meaneduc_nan.iterrows():
        idx=row[0]
        idhogar=row[1]['idhogar']
        m=me[me['idhogar']==idhogar]['escolari'].tolist()[0]
        data.at[idx, 'meaneduc']=m
        data.at[idx, 'SQBmeaned']=m*m
        
    return data

In [31]:
train = data_cleaning(train)
test = data_cleaning(test)

### Extract heads of household

In [32]:
train = train.query('parentesco1==1')
train = train.drop('parentesco1', axis=1)
test = test.drop('parentesco1', axis=1)
print(train.shape)

(2973, 141)


## Convert one-hot variables into numeric
* 'epared', 'etecho', 'eviv' and 'instlevel' can be converted into numeric
*  like (bad, regular, good) -> (0 ,1, 2)

In [33]:
def get_numeric(data, status_name):
    # make a list of column names containing 'sataus_name'
    status_cols = [s for s in data.columns.tolist() if status_name in s]
    print('status column names')
    print(status_cols)
    # make a DataFrame with only status_cols
    status_df = data[status_cols]
    # change its column name like ['epared1', 'epared2', 'epared3'] -> [0, 1, 2]
    status_df.columns = list(range(status_df.shape[1]))
    # get the column name which has the biggest value in every row
    
    # this is pandas.Series
    status_numeric = status_df.idxmax(1)
    # set Series name
    status_numeric.name = status_name
    # add status_numeric as a new column
    data = pd.concat([data, status_numeric], axis=1)
    return data

In [34]:
status_name_list = ['epared', 'etecho', 'eviv', 'instlevel']
for status_name in status_name_list:
    train = get_numeric(train, status_name)
    test = get_numeric(test, status_name)

status column names
['epared1', 'epared2', 'epared3']
status column names
['epared1', 'epared2', 'epared3']
status column names
['etecho1', 'etecho2', 'etecho3']
status column names
['etecho1', 'etecho2', 'etecho3']
status column names
['eviv1', 'eviv2', 'eviv3']
status column names
['eviv1', 'eviv2', 'eviv3']
status column names
['instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9']
status column names
['instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9']


## Delete needless columns
### redundant columns
* r4t3, tamviv, tamhog, hhsize ... almost the same as hogar_total
* v14a ... almost the same as saniatrio1
* v18q, mobilephone ... can be generated by v18q1, qmobilephone
* SQBxxx, agesq ... squared values
* parentescoxxx ... only heads of household are in dataset now

### extra columns
(One-hot variables should be linearly independent. For example, female (or male) column is needless, because whether the sample is female or not can be explained only with male (or female) column.)
* paredother, pisoother, abastaguano, energcocinar1, techootro, sanitario6, elimbasu6, estadocivil7, parentesco12, tipovivi5, lugar1, area1, female

### obsolete columns
* epared1~3, etecho1~3, eviv1~3, instlevel1~9 ... we don't use these columns anymore.


In [35]:
needless_cols = ['r4t3', 'tamhog', 'tamviv', 'hhsize', 'v18q', 'v14a', 'agesq',
                 'mobilephone', 'paredother', 'pisoother', 'abastaguano',
                 'energcocinar1', 'techootro', 'sanitario6', 'elimbasu6',
                 'estadocivil7', 'parentesco12', 'tipovivi5',
                 'lugar1', 'area1', 'female', 'epared1', 'epared2',
                 'epared3', 'etecho1', 'etecho2', 'etecho3',
                 'eviv1', 'eviv2', 'eviv3', 'instlevel1', 'instlevel2',
                 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6',
                 'instlevel7', 'instlevel8', 'instlevel9']
SQB_cols = [s for s in train.columns.tolist() if 'SQB' in s]
parentesco_cols = [s for s in train.columns.tolist() if 'parentesco' in s]

needless_cols.extend(SQB_cols)
needless_cols.extend(parentesco_cols)

train = train.drop(needless_cols, axis=1)
test = test.drop(needless_cols, axis=1)

In [36]:
ori_train = pd.read_csv(os.path.join(input_dir, 'train.csv'))
ori_train_X = ori_train.drop(['Id', 'Target', 'idhogar'], axis=1)

train_X = train.drop(['Id', 'Target', 'idhogar'], axis=1)

print('feature columns \n {} -> {}'.format(ori_train_X.shape[1], train_X.shape[1]))

feature columns 
 140 -> 85


## Simple LightGBM

In [37]:
# Split data
train_Id = train['Id'] # individual ID
train_idhogar = train['idhogar'] # household ID
train_y = train['Target'] # Target value
train_X = train.drop(['Id', 'Target', 'idhogar'], axis=1) # features

test_Id = test['Id'] # individual ID
test_idhogar = test['idhogar'] # household ID
test_X = test.drop(['Id', 'idhogar'], axis=1) # features

# Union train and test
all_Id = pd.concat([train_Id, test_Id], axis=0, sort=False)
all_idhogar = pd.concat([train_idhogar, test_idhogar], axis=0, sort=False)
all_X = pd.concat([train_X, test_X], axis=0, sort=False)

In [38]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score, make_scorer
import lightgbm as lgb

X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.1, random_state=0)

F1_scorer = make_scorer(f1_score, greater_is_better=True, average='macro')

# gbm_param = {
#     'num_leaves':[210]
#     ,'min_data_in_leaf':[9]
#     ,'max_depth':[14]
# }
# gbm = GridSearchCV(
#     lgb.LGBMClassifier(objective='multiclassova', class_weight='balanced', seed=0)
#     , gbm_param
#     , scoring=F1_scorer
# )


# params = {'num_leaves': 13, 'min_data_in_leaf': 23, 'max_depth': 11, 'learning_rate': 0.09, 'feature_fraction': 0.74}
gbm = lgb.LGBMClassifier(boosting_type='dart', objective='multiclassova', class_weight='balanced', random_state=0)
# gbm.set_params(**params)

gbm.fit(X_train, y_train)
# gbm.best_params_

LGBMClassifier(boosting_type='dart', class_weight='balanced',
        colsample_bytree=1.0, learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31,
        objective='multiclassova', random_state=0, reg_alpha=0.0,
        reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0)

In [39]:
import pickle
with open(os.path.join(working_dir, '20180801_lgbm.pickle'), mode='wb') as f:
    pickle.dump(gbm, f)

In [40]:
y_test_pred = gbm.predict(X_test)
cm = confusion_matrix(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred, average='macro')
print("confusion matrix: \n", cm)
print("macro F1 score: \n", f1)

confusion matrix: 
 [[  7   8   2   5]
 [  5  18  12  13]
 [  5  16   8   9]
 [  4  18  20 148]]
macro F1 score: 
 0.41746840819793984


In [41]:
pred = gbm.predict(test_X)
pred = pd.Series(data=pred, index=test_Id.values, name='Target')
pred = pd.concat([test_Id, pred], axis=1, join_axes=[test_Id.index])
pred.to_csv('submission.csv', index=False)